In [ ]:
# This notebook is trying out different pipeline approaches

# Cropped data is used here (5 minutes only), tried on whole data - takes forever.


#Load data, make folders
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mne
import pyprep as pp


In [ ]:

#from Functions.main_meg_qc import initial_stuff
from data_load_and_folders import load_meg_data
duration=5 #in minutes
#n_events, df_epochs_mags, df_epochs_grads, epochs_mags, epochs_grads, mags, grads, filtered_d, filtered_d_resamp, raw_cropped, raw=initial_stuff(duration)

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003483/sub-009/ses-1/meg/sub-009_ses-1_task-deduction_run-1_meg.fif'
#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003352/sub-1/ses-01/meg/sub-1_ses-01_task-ColorSpirals_run-00_meg.fif'
#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003352/sub-4/ses-02/meg/sub-4_ses-02_task-ColorSpirals_run-01_meg.fif'
#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003392/sub-01/meg/sub-01_task-localizer_meg.fif'
#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003645/sub-002/meg/sub-002_task-FacePerception_run-1_meg.fif'
data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003682/sub-001/ses-01/meg/sub-001_ses-01_task-AversiveLearningReplay_run-01_meg.fif'


raw, channels = load_meg_data(data_file)

#crop the data to calculate faster
raw_cropped = raw.copy()
#raw_cropped.crop(tmin=1100, tmax=None) 

raw_cropped



In [ ]:
# Annotate bad peak to peak amplitudes: peaks and flats
# USE TO MARK WHOLE CHANNELS AS BAD

# https://mne.tools/stable/generated/mne.preprocessing.annotate_amplitude.html
# copmpare this MNE function with what I wrote myself: Funnks - > Peaks_meg_qc -> neighbour_peak_amplitude

#'Creates annotations BAD_peak or BAD_flat for spans of data where consecutive samples exceed the threshold in peak or fall below the threshold 
# in flat for more than min_duration' 

amplit_annot=mne.preprocessing.annotate_amplitude(raw_cropped, peak={'mag':4e-14, 'grad': 4e-14}, flat=3e-14, bad_percent=5, min_duration=0.002, verbose=True)

#  !!! Automatically choose peak and flat values by averaging the data maybe?


# Some input parameters:
# * bad_percent - percent of tolerated bad segments in the channel. id some channel is over this percent - will be returned as "bads". 
#   While if its under, just the points will be returned as "bad peak or "bad flat"
#   However I dont see any info about which channel these points belong to! Which is totally stupid?
# * picks can call channel types or separate channel names like: picks=['grad'] or picks=['MEG0111']
# * alterbnatively u can specify peaks and/or flats values for types of channels like: peak={'mag':3.5e-11, 'grad': 4e-11} (but not the names of channels).
# * min_duration is in minutes
# * peaks and flats are in the same scale as the data, like: 1e-13

# Differences with my function in Funks.Peaks_meg_gc.ipynb: 
# - no option to set distance between up and down peak to be concedered a pair. They calculate peak like: abs(a[i+1] - a[i]) ≥ peak (from docs).
#   So we can probably adjust indexing of 'a' here to decide which peaks concider as an up+down pair.
# - This function need raw obj as input. So not epochs and not just a piece of data -> need to adjust it for epochs. 
# - Can set here extra: flat, percent of bad, min duration and actual height of peaks and flats.

# Now need to add these annotation obj to the raw:
raw_cropped.set_annotations(raw_cropped.annotations + amplit_annot[0])  
raw_cropped.info['bads'] = amplit_annot[1]
# annotate_amplitude creates a tuple, not just annotations, like other functions from this series. 
# Second part of tuple is 'bads' (whole channel is bad, too many bad segments). 
# This whole tuple can't be added to annots in raw, need to slice the first part like: [0]


#print('\nAnnotations added to raw:', raw_cropped.annotations)
#print('Channels added to bads in raw:', raw_cropped.info['bads'],"\n")


# ANNOTATIONS ARE SUMMED HERE. SO IF YOU WANT TO OVERWRITE THEM, RUN ALL CELLS ABOVE AGAIN OR JUST DELETE THE ANNOTATIONS like: 
# idx=[a for a in range(0,len(raw_cropped.annotations))] #all annotations chosen for removal
# raw_cropped.annotations.delete(idx)
# ONLY RERUNNING THIS ONE CELL WILL NOT REMOVE OLD ANNOTS FROM RAW. 

# raw_cropped.plot() #plot the data with annotations

sid='009'
# export annotation to dataframe:
df_annot=raw_cropped.annotations.to_data_frame()
df_annot.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/ptp_amplitude_annots_all.csv')
df_annot


In [ ]:
# Since this function doesnt give channel names, need to loop over every channel and add its name into every annotation.
# mne.Annotations obj doesnt allow changing, so will have to create a new object and write explicitly all info + channel name there.

# USE TO MARK EPOCH AS BAD, EXTRACT TIME OF FLAT/PEAK + WHICH CHANNEL

def get_amplitude_annots_per_channel(raw: mne.io.Raw, peak: float, flat: float, ch_type_names: list) -> tuple[pd.DataFrame, list]:
    """Function creates amplitude (peak-to-peak annotations for every channel separately"""
    
    amplit_annot_with_ch_names=mne.Annotations(onset=[], duration=[], description=[], orig_time=raw.annotations.orig_time) #initialize 
    bad_channels=[]

    for channel in ch_type_names:
        #get annotation object:
        amplit_annot=mne.preprocessing.annotate_amplitude(raw, peak=peak, flat=flat , bad_percent=5, min_duration=0.002, picks=[channel], verbose=False)
        
        bad_channels.append(amplit_annot[1]) #Can later add these into annotation as well.

        if len(amplit_annot[0])>0:

            #create new annot obj and add there all data + channel name:
            amplit_annot_with_ch_names.append(onset=amplit_annot[0][0]['onset'], duration=amplit_annot[0][0]['duration'], description=amplit_annot[0][0]['description'], ch_names=[[channel]])

    df_ptp_amlitude_annot=amplit_annot_with_ch_names.to_data_frame()

    return df_ptp_amlitude_annot, bad_channels, amplit_annot_with_ch_names




In [ ]:
# # clear annotations:
# idx=[a for a in range(0,len(raw_cropped.annotations))] #all annotations chosen for removal
# raw_cropped.annotations.delete(idx)

peak_m=4e-14
peak_g=4e-14
flat=3e-14
bad_percent = 5
min_duration = 0.002

raw_cropped = raw.copy()
raw_cropped.crop(tmin=1100, tmax=None) 
# annotate amplitude per channel:
#df_ptp_amlitude_annot_mags, bad_channels, amplit_annot_with_ch_names_mags=get_amplitude_annots_per_channel(raw_cropped, peak_m, flat, ch_type_names=channels['mags'])
#df_ptp_amlitude_annot_grads, bad_channels, amplit_annot_with_ch_names_grads=get_amplitude_annots_per_channel(raw_cropped, peak_g, flat, ch_type_names=channels['grads'])
#amplit_annot_with_ch_names_grads=get_amplitude_annots_per_channel(raw_cropped, peak, flat, ch_type_names=grads)

from Peaks_auto_meg_qc import get_amplitude_annots_per_channel
df_ptp_amlitude_annot_mags, bad_channels_mags=get_amplitude_annots_per_channel(raw_cropped, peak_m, flat, channels['mags'],bad_percent, min_duration)
df_ptp_amlitude_annot_mags.to_csv('/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/derivatives/sub-'+sid+'/megqc/csv files/ptp_amplitude_annots_mags.csv')

df_ptp_amlitude_annot_mags, bad_channels_mags=get_amplitude_annots_per_channel(raw_cropped, peak_m, flat, channels['grads'],bad_percent, min_duration)
df_ptp_amlitude_annot_mags.to_csv('/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/derivatives/sub-'+sid+'/megqc/csv files/ptp_amplitude_annots_grads.csv')

df_ptp_amlitude_annot_mags


In [ ]:
# Annotate breaks - doesnt work..

break_annots = mne.preprocessing.annotate_break(
    raw=raw_cropped,
    min_break_duration=5,  # consider segments of at least 5 s duration
    t_start_after_previous=2,  # start annotation 2 s after end of previous one
    t_stop_before_next=2  # stop annotation 2 s before beginning of next one
)


raw_cropped.set_annotations(raw_cropped.annotations + break_annots)  # add to existing
raw_cropped.plot()

In [ ]:
# Annotate movement - like in tutorial 2 from:
# https://mne.tools/stable/generated/mne.preprocessing.annotate_movement.html

from mne.preprocessing import annotate_movement, compute_average_dev_head_t

# Get cHPI time series and compute average
chpi_locs = mne.chpi.extract_chpi_locs_ctf(raw)
head_pos = mne.chpi.compute_head_pos(raw.info, chpi_locs)
original_head_dev_t = mne.transforms.invert_transform(
    raw.info['dev_head_t'])
average_head_dev_t = mne.transforms.invert_transform(
    compute_average_dev_head_t(raw, head_pos))
fig = mne.viz.plot_head_positions(head_pos)
for ax, val, val_ori in zip(fig.axes[::2], average_head_dev_t['trans'][:3, 3],
                            original_head_dev_t['trans'][:3, 3]):
    ax.axhline(1000 * val, color='r')
    ax.axhline(1000 * val_ori, color='g')

# The green horizontal lines represent the original head position, whereas the
# red lines are the new head position averaged over all the time points.

mean_distance_limit = 0.0015  # in meters
annotation_movement, hpi_disp = annotate_movement(
    raw, head_pos, mean_distance_limit=mean_distance_limit)
raw.set_annotations(annotation_movement)
raw.plot(n_channels=100, duration=20)

In [ ]:
#Since we got an error, try another tutorial: 
# (takes some minutes depends how big is raw_cropped: 2-40 min):
# https://mne.tools/stable/auto_tutorials/preprocessing/59_head_positions.html#sphx-glr-auto-tutorials-preprocessing-59-head-positions-py

# cHPI - continuous head position indicator (HPI) coil channels, data in teslas

# 'We can use mne.chpi.get_chpi_info to retrieve the coil frequencies, the index of 
# the channel indicating when which coil was switched on, and the respective “event codes” 
# associated with each coil’s activity.'
chpi_freqs, ch_idx, chpi_codes = mne.chpi.get_chpi_info(info=raw_cropped.info, on_missing='warn', verbose=None)
# Output:
# - The frequency used for each individual cHPI coil.
# - The index of the STIM channel containing information about when which cHPI coils were switched on.
# - The values coding for the “on” state of each individual cHPI coil.

# The values coding for the “on” state of each individual cHPI coil.
print(f'cHPI coil frequencies extracted from raw: {chpi_freqs} Hz')

#We only got 5, not 9 HPI (see error in cell above)


#extract the HPI coil amplitudes as a function of time:
print('Extract the HPI coil amplitudes as a function of time:')
chpi_amplitudes=mne.chpi.compute_chpi_amplitudes(raw_cropped)
#chpi_amplitudes=mne.chpi.compute_chpi_amplitudes(raw_cropped, t_step_min=0.01, t_window='auto', ext_order=1, tmin=0, tmax=None, verbose=None)


#compute time-varying HPI coil locations from these
print('Compute time-varying HPI coil locations from these')
#chpi_locs=mne.chpi.compute_chpi_locs(raw_cropped.info, chpi_amplitudes, t_step_max=1.0, too_close='raise', adjust_dig=False, verbose=None)
chpi_locs=mne.chpi.compute_chpi_locs(raw_cropped.info, chpi_amplitudes)


print('Compute head positions from the coil locations:')
#compute head positions from the coil locations:
head_pos = mne.chpi.compute_head_pos(raw_cropped.info, chpi_locs, verbose=True)
print('head_positions computed:', head_pos)



In [ ]:
#Visualizing continuous head position: doesnt work

mne.viz.plot_head_positions(head_pos, mode='traces')
#mne.viz.plot_head_positions(head_pos, mode='field')



In [ ]:
# Now again try to annotate movement: also error - It didnt calculate any positions!

mean_distance_limit = 0.0015  # in meters
annotation_movement, hpi_disp = annotate_movement(
    raw_cropped, head_pos, mean_distance_limit=mean_distance_limit)
raw_cropped.set_annotations(raw_cropped.annotations + annotation_movement)
raw_cropped.plot(n_channels=100, duration=20)

In [ ]:
# https://pyprep.readthedocs.io/en/latest/generated/pyprep.NoisyChannels.html#pyprep.NoisyChannels

# This class implements all of the noisy channel detection methods used in the PREP pipeline, as described in:
# Bigdely-Shamlo, N., Mullen, T., Kothe, C., Su, K. M., Robbins, K. A. (2015). The PREP pipeline: 
# standardized preprocessing for large-scale EEG analysis. Frontiers in Neuroinformatics, 9, 16.

noisy = pp.NoisyChannels(raw) #first, add raw to class, then all the noisy channels finding performed on this new object.

# Doesnt work. Only EEG is supported. Tried to change channel type in just to try in:
# opt/anaconda3/envs/mne_new/lib/python3.9/site-packages/pyprep/find_noisy_channels.py
# to self.raw_mne.pick_types(meg=True, verbose=True)
# no luck, it gives traceback on this line. 
# (If u explicitely call raw.pick_types(meg=True, verbose=True) - everything works fine - see cell above).

# Dont know where the issue is exactly, but in the end we ll need to rewrite this whole function 
# to work with MEG if we really need it.

from time import perf_counter

start_time = perf_counter()
#noisy.find_bad_by_ransac(channel_wise=True)
noisy.find_all_bads(ransac=True, channel_wise=False, max_chunk_size=None)
print("--- %s seconds ---" % (perf_counter() - start_time))